In [65]:
import pandas as pd
import re
from unidecode import unidecode
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import WhitespaceTokenizer
from nltk.stem import SnowballStemmer
from nltk.stem import WordNetLemmatizer

In [66]:
import spacy
nlp = spacy.load("es_dep_news_trf")
import es_dep_news_trf
nlp = es_dep_news_trf.load()

In [67]:
data_pe = pd.read_csv('C:\\Users\\kendr\\OneDrive\\Escritorio\\train\\pe.tsv',sep='\t', names=["Id", "Text", "Etiqueta"])
data_mx = pd.read_csv('C:\\Users\\kendr\\OneDrive\\Escritorio\\train\\mx.tsv',sep='\t', names=["Id", "Text", "Etiqueta"])
data_es = pd.read_csv('C:\\Users\\kendr\\OneDrive\\Escritorio\\train\\es.tsv',sep='\t', names=["Id", "Text", "Etiqueta"])
data_cr = pd.read_csv('C:\\Users\\kendr\\OneDrive\\Escritorio\\train\\cr.tsv',sep='\t', names=["Id", "Text", "Etiqueta"])
data_uy = pd.read_csv('C:\\Users\\kendr\\OneDrive\\Escritorio\\train\\uy.tsv',sep='\t', names=["Id", "Text", "Etiqueta"])

In [68]:
data_pe['Pais'] = 'PE'

In [69]:
data_mx['Pais'] = 'MX'

In [70]:
data_es['Pais'] = 'ES'

In [71]:
data_cr['Pais'] = 'CR'

In [72]:
data_uy['Pais'] = 'UY'

---------

Unión de los datos

In [73]:
data_train = pd.concat([data_pe, data_mx, data_es, data_cr, data_uy], ignore_index=True)

In [74]:
data_train.sample(5)

,Id,Text,Etiqueta,Pais
1260,817398806375374848,Estoy malita de la gripa,N,MX
3883,785692907802165248,"@cuervotinelli Buenas noches que descanses !!,...",P,UY
2856,765014135130259456,Es que es taaaaaan bonito,P,ES
3521,806726824608702464,Les juro que lo stalkeo y mi instinto asesino ...,N,CR
3745,797825741144686592,Vean los nuevos emojis #iOS102 El aguacate 🥑,NEU,CR


-------------

Limpieza de los datos

In [75]:
def get_clean_text(df, column):
    df[column] = df[column].apply(lambda elem: str.lower((elem)))
    
    #Elimina todos los acentos
    df[column] = df[column].apply(lambda elem: unidecode(elem))
    
    #Eliminar @#
    df[column] = df[column].apply(lambda elem: re.sub(r'[@#]\w+\s?', '', elem))
    
    #
    df[column] = df[column].apply(lambda elem: re.sub(r'[^a-zA-Z]', ' ', elem))
    
    #Eliminar jajaja
    df[column] = df[column].apply(lambda elem: re.sub(r"(jaja)+", '', elem, flags=re.IGNORECASE))
    
    #Eliminar hahaha
    df[column] = df[column].apply(lambda elem: re.sub(r"(haha)+", '', elem, flags=re.IGNORECASE))
    
    # Quitar punto entre abreviaturas
    df[column] = df[column].apply(lambda elem: re.sub(r'(\w)\.(\w)\.', r'\1\2\2', elem))
    
    # Eliminar Links
    df[column] = df[column].apply(lambda elem: re.sub(r'https?://\S+', '', elem))
    
    #Pasar numero a representación en palabras
    #df[column] = df[column].apply(lambda elem: num2words(elem, lang='es'))
    
    return df

In [76]:
get_clean_text(data_train, 'Text')

,Id,Text,Etiqueta,Pais
0,775087224857567232,lo que mas amo de mi escritorio es que hay una...,P,PE
1,778485882647089152,ese momento en el que no puedes dormir y no sa...,NEU,PE
2,774619666505400322,aw recien veo esto sorrry tu pues me dice...,NEU,PE
3,802359837308887040,soy el perro asesinado,N,PE
4,812398630137012224,ay esas medias tan lindas,P,PE
...,...,...,...,...
4797,790619641659944961,que tipo de proyecto de diseno es tu preferid...,NEU,UY
4798,785432238783619072,que bueno ver a personas que te alegran aunque...,P,UY
4799,776184535771906048,gracias queridisimo buenisimos la charla la...,P,UY
4800,795705345050025984,pero antes si era asi es parte de nuestro barr...,N,UY


---------

Tokenizar palabras

In [77]:
def get_tokenized_text(df, column):
    sw = set(stopwords.words('spanish'))
    nlp = spacy.load('es_core_news_sm')
    df[column] = df[column].apply(lambda text: ' '.join([token.lemma_ for token in nlp(text) if token.text.lower() not in sw]))
    return df

In [78]:
# Aplicar tokenización de palabras vacías a la columna 'Text'
df_tokenized = get_tokenized_text(data_train, 'Text')

df_tokenized.sample(10)

,Id,Text,Etiqueta,Pais
744,809864399594881025,creer mas jodido pasar listo venir jod...,P,PE
865,809324482053619712,genial incomunicado desconectado mundo veces,P,PE
1730,815114533119393792,olvidenlir ver si querer comprar costar ...,NEU,MX
1608,805525886116175873,si siempre decir navidad tratar solo regalo ...,N,MX
988,819000467577446401,puta madre faltar enfermar yo,N,MX
4051,766712913721712640,hoy intentar hablar cannabis boca mia incomodo...,N,UY
3533,776197117253607424,ahi si sabria decir tú generalmente siempre ha...,NEU,CR
3286,797640962952622080,charita canto igual,N,CR
2355,781215938632966144,hoy ver guapo,P,ES
1260,817398806375374848,malito gripa,N,MX


In [105]:
df_tokenized_copy = df_tokenized

Para la Clusterización unir todos los datos

In [79]:
from gensim.corpora import Dictionary
from gensim.models import HdpModel

In [80]:
pre_corpus = [text.split(' ') for text in df_tokenized.Text.to_list()]

In [81]:
pre_corpus

[['mas',
  'amar',
  'escritorio',
  'foto',
  'mia',
  'junto',
  'aparte',
  'cuarto',
  'cuadro',
  'tambien',
  'estan'],
 ['momento',
  'poder',
  'dormir',
  'saber',
  '',
  '',
  'sonrisa',
  'oreja',
  'oreja',
  'igual',
  'todo',
  'semana'],
 ['aw',
  '',
  '',
  '',
  'recien',
  'ver',
  '',
  '',
  'sorrry',
  '',
  '',
  '',
  'pues',
  'decir',
  'luego',
  'quedar',
  'ir',
  'hoy',
  'reu',
  'prima',
  '',
  '',
  'decir',
  'mari'],
 ['perro', 'asesinado'],
 ['ay', '', '', 'media', 'tanto', 'lindo'],
 ['histerico', 'bestia', '', '', 'pobre', 'bello'],
 ['quinto', 'dia', '', '', '', 'previo', 'restobar', 'pub'],
 ['necesitar', 'urgente', 'entrar', 'gimnasia', 'gimnasioo'],
 ['peor', '', '', 'querer', 'mejor'],
 ['mal',
  'momento',
  'tomar',
  'cafe',
  'caliente',
  '',
  '',
  'dejar',
  'sudar',
  '',
  '',
  'cono'],
 ['nunca', 'habiar', 'dormido', 'pelicula', 'timburton', 'raro'],
 ['hoy', 'sonar', 'profe', 'actuacion', 'extremadamente', 'feliz'],
 ['apagon', 

In [82]:
dictionary = Dictionary(pre_corpus)

In [83]:
dictionary.compactify()
# Filter extremes
dictionary.filter_extremes(no_below=2, no_above=0.97, keep_n=None)
dictionary.compactify()

In [84]:
corpus = [dictionary.doc2bow(text.split(" ")) for text in df_tokenized.Text.to_list()]

In [85]:
hdp = HdpModel(corpus=corpus, id2word=dictionary)

In [86]:
hdp.print_topics(num_topics=10, num_words=6)

[(0, '0.193* + 0.004*si + 0.003*ir + 0.003*hacer + 0.003*ver + 0.002*dia'),
 (1, '0.062* + 0.003*subi + 0.002*pp + 0.002*hacer + 0.002*yo + 0.002*mano'),
 (2,
  '0.039* + 0.002*bloquear + 0.002*literalmente + 0.002*si + 0.002*permiso + 0.002*jefe'),
 (3,
  '0.025* + 0.003*dios + 0.002*mas + 0.002*secreto + 0.002*dato + 0.002*examen'),
 (4,
  '0.017* + 0.003*reflejo + 0.003*creer + 0.002*debi + 0.002*euros + 0.002*toledo'),
 (5,
  '0.014* + 0.003*acostumbrar + 0.002*noche + 0.002*novio + 0.002*mega + 0.002*jalir'),
 (6,
  '0.016* + 0.003*lpm + 0.003*devolver + 0.002*relleno + 0.002*uber + 0.002*pie'),
 (7,
  '0.013* + 0.003*comerme + 0.002*lejano + 0.002*juntito + 0.002*estrategia + 0.002*amoroso'),
 (8,
  '0.012* + 0.003*argentino + 0.002*pro + 0.002*enfadado + 0.002*totalmente + 0.002*mimoso'),
 (9,
  '0.007* + 0.002*esperanza + 0.002*aspecto + 0.002*jurar + 0.002*masculino + 0.002*pleno')]

In [87]:
from gensim.models import CoherenceModel

In [88]:
CoherenceModel(model=hdp, texts=pre_corpus, dictionary=dictionary, coherence='c_v').get_coherence()

0.7172946385018015

-------------

Modelo **LDA**

In [89]:
from gensim.models import LsiModel
import gensim
from gensim import corpora
from itertools import product

In [90]:
num_topics = 10
lda_model = gensim.models.LdaModel(corpus, num_topics=num_topics, id2word=dictionary, passes=10)

In [91]:
topics = lda_model.print_topics(num_topics=num_topics, num_words=6)
for topic in topics:
    print(topic)

(0, '0.041*"nadie" + 0.037*"mirar" + 0.029*"junto" + 0.026*"pasado" + 0.020*"ir" + 0.019*"peor"')
(1, '0.033*"extrano" + 0.021*"hacer" + 0.021*"decir" + 0.020*"foto" + 0.020*"seguir" + 0.017*"ver"')
(2, '0.114*"mejor" + 0.026*"buen" + 0.023*"poder" + 0.022*"hoy" + 0.022*"mundo" + 0.021*"dia"')
(3, '0.040*"ver" + 0.031*"alguien" + 0.029*"necesitar" + 0.024*"ir" + 0.023*"dejar" + 0.021*"yo"')
(4, '0.041*"siempre" + 0.038*"sentir" + 0.032*"persona" + 0.032*"q" + 0.031*"mas" + 0.027*"pensar"')
(5, '0.046*"tanto" + 0.033*"dar" + 0.033*"triste" + 0.026*"mas" + 0.022*"cosa" + 0.021*"segundo"')
(6, '0.040*"hacer" + 0.021*"q" + 0.015*"hermoso" + 0.015*"asi" + 0.015*"ver" + 0.013*"nuevo"')
(7, '0.045*"ir" + 0.043*"él" + 0.041*"mismo" + 0.036*"dormir" + 0.031*"pasar" + 0.026*"poner"')
(8, '0.514*"" + 0.011*"si" + 0.009*"hacer" + 0.009*"ir" + 0.006*"dia" + 0.006*"mas"')
(9, '0.050*"solo" + 0.035*"bonito" + 0.031*"ir" + 0.027*"feliz" + 0.021*"querer" + 0.020*"hoy"')


In [92]:
CoherenceModel(model=lda_model, texts=pre_corpus, dictionary=dictionary, coherence='c_v').get_coherence()

0.36472356873688944

---------------

Modelo **LSI**

In [93]:
lsi_model = gensim.models.LsiModel(corpus, id2word=dictionary, num_topics=10)

In [94]:
topics = lsi_model.print_topics(num_topics=10, num_words=6)
for topic in topics:
    print(topic)

(0, '0.999*"" + 0.017*"si" + 0.014*"ir" + 0.014*"hacer" + 0.010*"dia" + 0.010*"ver"')
(1, '-0.614*"ir" + -0.452*"si" + -0.282*"hacer" + -0.282*"ver" + -0.180*"querer" + -0.160*"mas"')
(2, '-0.698*"ir" + 0.634*"si" + 0.174*"hacer" + 0.163*"ver" + 0.132*"mas" + -0.073*"ser"')
(3, '-0.735*"hacer" + 0.511*"si" + 0.229*"ir" + -0.179*"mas" + -0.135*"él" + -0.129*"poder"')
(4, '-0.652*"ver" + 0.472*"hacer" + -0.315*"mas" + 0.304*"si" + -0.160*"q" + -0.157*"querer"')
(5, '0.748*"mas" + -0.529*"ver" + -0.199*"hacer" + 0.157*"dia" + -0.139*"él" + 0.139*"ser"')
(6, '0.731*"dia" + 0.515*"buen" + -0.244*"querer" + -0.223*"mas" + 0.133*"hoy" + 0.086*"q"')
(7, '0.791*"q" + -0.289*"mas" + -0.285*"ver" + 0.168*"él" + 0.159*"querer" + -0.144*"hacer"')
(8, '-0.466*"querer" + -0.463*"poder" + 0.453*"q" + 0.267*"mas" + -0.219*"solo" + -0.200*"ser"')
(9, '0.676*"querer" + -0.606*"poder" + 0.160*"solo" + -0.145*"mejor" + -0.139*"ano" + 0.131*"dia"')


In [95]:
CoherenceModel(model=lsi_model, texts=pre_corpus, dictionary=dictionary, coherence='c_v').get_coherence()

0.3308505600945233

In [96]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn import metrics

In [97]:
# Vectorización
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df_tokenized['Text'])

# Aplicar K-means
k = 4  # Número de clusters deseado
kmeans = KMeans(n_clusters=k)
kmeans.fit(X)

# Agrega esta linea a kmeans
clusters = kmeans.labels_.tolist()

# Obtener etiquetas de los clusters
etiquetas = kmeans.labels_

# Evaluación de resultados
inercia = kmeans.inertia_
coef_silueta = metrics.silhouette_score(X, etiquetas)

C:\Users\kendr\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning



In [98]:
coef_silueta

0.003851424014999927

In [99]:
import matplotlib.pyplot as plt
from sklearn.decomposition import TruncatedSVD
import plotly.express as px

# Reducción de dimensionalidad con TruncatedSVD
svd = TruncatedSVD(n_components=2)
X_svd = svd.fit_transform(X)

# Crear DataFrame con los datos y las etiquetas
df = pd.DataFrame({'Componente 1': X_svd[:, 0], 'Componente 2': X_svd[:, 1], 'Etiquetas': etiquetas})

# Graficar los clusters con plotly express
fig = px.scatter(df, x='Componente 1', y='Componente 2', color='Etiquetas',
                 labels={'Componente 1': 'Componente 1', 'Componente 2': 'Componente 2'},
                 title='Clusters con K-means')

# Mostrar la figura
fig.show()

In [100]:
tweets = {'Tweet': df_tokenized['Text'].tolist(), 'Cluster': clusters}
df = pd.DataFrame(tweets, index = [clusters])
df.sample(10)

,Tweet,Cluster
2,centro comercial,2
3,horrible buen devastadorar,3
0,decir crees caer mal terapia ver cambi...,0
2,acabar tragar pastel mood celín,2
0,poder ver mariconada,0
2,supuesto importar nadie hombre eurodescend...,2
0,aunque intentar ver episodio seguido acabar,0
2,pesar limitación humano senor querer obrar m...,2
2,noemi duer yo abandonado hacedme caso,2
0,vez dejar mamar verdad ver tanto guapo,0


In [120]:
df_combinado = pd.merge(df, df_tokenized_copy, left_index=True, right_index=True)
df_combinado = df_combinado[['Id', 'Tweet', 'Cluster', 'Etiqueta_Predicha']]
df_combinado.head()

TypeError: '<' not supported between instances of 'int' and 'tuple'

In [104]:
# Calcular la cantidad de cada polaridad en cada cluster
conteo_polaridad = df_combinado.groupby(['Cluster', 'Etiqueta_Predicha'])['Id'].count().unstack()

# Determinar qué polaridad influye más en cada cluster
polaridad_influyente = conteo_polaridad.idxmax(axis=1)

# Imprimir los resultados
for cluster, polaridad in polaridad_influyente.items():
    print(f"Cluster {cluster}: Tópico influyente: {polaridad}")

NameError: name 'df_combinado' is not defined

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

# Crear el vectorizador de frecuencia de términos
vectorizer = CountVectorizer()

# Calcular la matriz de frecuencia de términos
X_freq = vectorizer.fit_transform(df['Tweet'])

# Obtener los términos
terms = vectorizer.get_feature_names_out()

# Reiniciar el índice del DataFrame df
df.reset_index(drop=True, inplace=True)

# Agregar la columna 'Cluster' al DataFrame df_freq
df_freq = pd.DataFrame(X_freq.toarray(), columns=terms)
df_freq = df_freq.assign(Cluster=df['Cluster'])  # Asignar columna 'Cluster' reiniciando el índice

# Calcular la suma de frecuencias de términos por cluster
df_cluster_freq = df_freq.groupby('Cluster').sum().reset_index()

# Obtener los términos más frecuentes en cada cluster
num_top_terms = 5  # Número de términos más frecuentes a mostrar

for cluster in df_cluster_freq['Cluster']:
    cluster_terms = df_cluster_freq[df_cluster_freq['Cluster'] == cluster].drop('Cluster', axis=1)
    top_terms = cluster_terms.squeeze().nlargest(num_top_terms)
    print(f"Cluster {cluster}:")
    print(top_terms)
    print()

In [ ]:
# Crear un gráfico de Word Cloud para cada cluster
for cluster in df_cluster_freq['Cluster']:
    cluster_terms = df_cluster_freq[df_cluster_freq['Cluster'] == cluster].drop('Cluster', axis=1)
    frequencies = cluster_terms.squeeze().to_dict()

    # Crear el objeto WordCloud
    wordcloud = WordCloud(background_color='black')

    # Generar el Word Cloud
    wordcloud.generate_from_frequencies(frequencies)

    # Mostrar el Word Cloud
    plt.figure(figsize=(15, 13))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis('off')
    plt.title(f"Cluster {cluster} - Términos más frecuentes", fontsize=20, color='white')
    plt.show()

In [ ]:
# Crear un gráfico de barras para mostrar los términos más frecuentes en cada cluster
df_cluster_freq_long = pd.melt(df_cluster_freq, id_vars='Cluster', value_vars=terms, var_name='Term', value_name='Frequency')
fig = px.bar(df_cluster_freq_long, x='Frequency', y='Term', color='Cluster', orientation='h')
fig.show()

In [ ]:
# Definir los rangos de frecuencia
rango_bajo = (0, 100)
rango_medio = (101, 200)
rango_alto = (201, 500)

# Filtrar los términos por rango de frecuencia
df_cluster_freq_filtered_bajo = df_cluster_freq_long[(df_cluster_freq_long['Frequency'] >= rango_bajo[0]) & (df_cluster_freq_long['Frequency'] <= rango_bajo[1])]
df_cluster_freq_filtered_medio = df_cluster_freq_long[(df_cluster_freq_long['Frequency'] >= rango_medio[0]) & (df_cluster_freq_long['Frequency'] <= rango_medio[1])]
df_cluster_freq_filtered_alto = df_cluster_freq_long[(df_cluster_freq_long['Frequency'] >= rango_alto[0]) & (df_cluster_freq_long['Frequency'] <= rango_alto[1])]

# Crear los gráficos de barras para cada rango de frecuencia
fig_bajo = px.bar(df_cluster_freq_filtered_bajo, x='Frequency', y='Term', color='Cluster', orientation='h', title='Frecuencia de términos (Rango: Bajo)')
fig_medio = px.bar(df_cluster_freq_filtered_medio, x='Frequency', y='Term', color='Cluster', orientation='h', title='Frecuencia de términos (Rango: Medio)')
fig_alto = px.bar(df_cluster_freq_filtered_alto, x='Frequency', y='Term', color='Cluster', orientation='h', title='Frecuencia de términos (Rango: Alto)')

# Aumentar el tamaño de los gráficos
fig_bajo.update_layout(height=600, width=800)
fig_medio.update_layout(height=600, width=800)
fig_alto.update_layout(height=600, width=800)

# Mostrar los gráficos
fig_bajo.show()
fig_medio.show()
fig_alto.show()

In [ ]:
df_cluster_freq_long.sort_values(by='Frequency').tail()

------------

Clasificar de los tweets como positivos, negativos o neutros según su 
contenido.

Se pueden utilizar técnicas como análisis de sentimientos basado en palabras 
clave, clasificadores de aprendizaje automático (por ejemplo, Naive Bayes) o 
modelos de procesamiento del lenguaje natural (por ejemplo, BERT).

Explicar la técnica utilizada y proporcionar el resultado de la clasificación.

In [109]:
# plotting
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

In [110]:
df_tokenized_copy = df_tokenized

Naive Bayes

In [119]:
X = df_tokenized_copy['Text']

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(X)

clf = MultinomialNB()
clf.fit(X, df_tokenized_copy['Etiqueta'])

y_pred = clf.predict(X)

accuracy = accuracy_score(df_tokenized_copy['Etiqueta'], y_pred)
print(f"Porcentaje de acierto: {accuracy}")

df_tokenized_copy['Etiqueta_Predicha'] = y_pred

Porcentaje de acierto: 0.8846314035818409


In [112]:
df_tokenized_copy.head()

,Id,Text,Etiqueta,Pais,Etiqueta_Predicha
0,775087224857567232,mas amar escritorio foto mia junto aparte cuar...,P,PE,P
1,778485882647089152,momento poder dormir saber sonrisa oreja ore...,NEU,PE,NEU
2,774619666505400322,aw recien ver sorrry pues decir luego ...,NEU,PE,NEU
3,802359837308887040,perro asesinado,N,PE,N
4,812398630137012224,ay media tanto lindo,P,PE,P


Calcular el porcentaje de diferencia por cada etiqueta &darr;

In [113]:
porcentaje_diferencia_por_etiqueta = {}

grupos_etiqueta = df_tokenized_copy.groupby('Etiqueta')

for etiqueta, grupo in grupos_etiqueta:
    porcentaje_diferencia = (grupo['Etiqueta'] != grupo['Etiqueta_Predicha']).mean() * 100
    porcentaje_diferencia_por_etiqueta[etiqueta] = porcentaje_diferencia

for etiqueta, porcentaje in porcentaje_diferencia_por_etiqueta.items():
    print(f"Etiqueta: {etiqueta}\tPorcentaje de diferencia: {porcentaje:.2f}%")

Etiqueta: N	Porcentaje de diferencia: 5.41%
Etiqueta: NEU	Porcentaje de diferencia: 19.83%
Etiqueta: P	Porcentaje de diferencia: 10.76%


Calcular el procentaje de cada etiqueta &darr;

In [114]:
porcentaje_etiquetas_predichas = (df_tokenized_copy['Etiqueta_Predicha'].value_counts() / len(df_tokenized_copy)) * 100

for etiqueta, porcentaje in porcentaje_etiquetas_predichas.items():
    print(f"Etiqueta predicha: {etiqueta}\tPorcentaje: {porcentaje:.2f}%")

Etiqueta predicha: N	Porcentaje: 43.17%
Etiqueta predicha: P	Porcentaje: 29.53%
Etiqueta predicha: NEU	Porcentaje: 27.30%


Porcentaje etiqueta antigua &darr;

In [115]:
porcentaje_etiquetas = (df_tokenized_copy['Etiqueta'].value_counts() / len(df_tokenized_copy)) * 100

for etiqueta, porcentaje in porcentaje_etiquetas.items():
    print(f"Etiquetas: {etiqueta}\tPorcentaje: {porcentaje:.2f}%")

Etiquetas: N	Porcentaje: 39.25%
Etiquetas: NEU	Porcentaje: 31.72%
Etiquetas: P	Porcentaje: 29.03%


In [116]:
import plotly.express as px

In [117]:
df_filtered = df_tokenized_copy[df_tokenized_copy['Etiqueta_Predicha'].isin(['P', 'N', 'NEU'])]

grouped = df_filtered.groupby(['Pais', 'Etiqueta_Predicha']).size().reset_index(name='Count')

total_registros = grouped.groupby('Pais')['Count'].sum().reset_index(name='Total')

# Calcular el porcentaje de cada etiqueta por país
grouped = grouped.merge(total_registros, on='Pais')
grouped['Percentage'] = round((grouped['Count'] / grouped['Total'] * 100), 2)

# Generar el gráfico de barras con Plotly Express
fig = px.bar(grouped, x='Pais', y='Percentage', color='Etiqueta_Predicha',
             title='Porcentaje de sentimiento por país',
             labels={'Pais': 'País', 'Percentage': 'Porcentaje', 'Etiqueta': 'Etiqueta'},
             barmode='stack')

# Mostrar el gráfico
fig.show()

In [ ]:
from PIL import ImageFont, ImageDraw
from wordcloud import WordCloud

In [ ]:
# Unir todos los textos en una sola cadena
text = ' '.join(df_tokenized_copy['Text'])

# Crear el objeto WordCloud especificando la fuente TrueType
wordcloud = WordCloud(width=800, height=400, max_words=100, background_color='white', font_path='./arial.ttf').generate(text)

# Mostrar el WordCloud
plt.figure(figsize=(10, 6))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.show()

--------------

TÉCNICA UTILIZADA

Las técnicas utilizadas son **CountVectorizer** y **MultinomialNB**, utilizadas en el procesamiento de lenguaje natural (NLP) y la clasificación de textos.

Se utiliza el **CountVectorizer** para convertir los textos en una representación numérica, creando una matriz de recuento de palabras.

Se entrena un clasificador **MultinomialNB** utilizando la matriz numérica y las etiquetas correspondientes del DataFrame.

Se realiza una predicción utilizando el mismo conjunto de datos de entrada.

Esta técnica permite entrenar un modelo para clasificar textos en categorías específicas basándose en la frecuencia de las palabras presentes en los textos.

2) Limpieza y preprocesado de los datos
 - Tweets 5 paises diferentes
 - Conjunto aleatorio de Tweets
 - Costa Rica: 777, España: 1126, Peru: 966, Mexico: 990, Uruguay: 943
 - Libreria Regex para el tratado de caracteres especiales
 - (@,# y url) han sido eliminados para este estudio
 - Todos los caracteres especiales como signos de puntuacion y acentos han sido eliminados
 - Todos los tweets han sido cinvertidos a minusculas
Es decir no hemos quedados solo con palabras y numero, excluyendo todo lo anterior nombrado

**Aporte**: Aunque se realizo un analisis exautivo para eliminar caracteres que lo entorpecian, se necesita mas tiempo para realizar un trabajo mas afondo para eliminar todo el ruido de manera satisfactoria.

Luego se ha realizado la tokenización y lematización.

La eliminación de las **Stopswords** no se realizó de la manera esperada debido al ruido anterior mente comentado.

La libreria no tuvo el desempeño esperado debido a la magnitud del ruido en los textos.

Se ha realizado Lematizacion y Estemización, debido a los resultado optenidos se ha obtado por utilizar lematización.

3) Clustering
 Kmeans: 
 - Gracias a los resultados en el Ebowlmetod se ha optado por elegir que el numero de clusters optimo es entre 4 y 6.
 - En la grafica se puede observar que en los clusters no se delimita bien su agrupación, por lo que concluimos que su interpretación se dificulta.
 
 Agglomerative:
 - Una diferencia notoria entre ambos modelos es que en agglomerative se observa una superposicion muy marcada en los clusters.
 
 HDP, NMF, LDA, LSI:
 - A pesar de que hay diferencias notorias en cada uno de los modelos (detalladas mas abajo), los resultados obtenidos no permiten sacar concluciones facilomente interpretables.
 - Se cree que se debe a ruido en los datos, falta de información acerca de los datos o eleccion incorrecta de lo modelos.
 
4) Metricas